In [3]:
# ----------------------------------------------------------------------------------------
import pickle   
from os import listdir
from os.path import isfile, join
import spacy #load spacy
nlp = spacy.load('en_core_web_sm')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random as rd
from collections import defaultdict
import matplotlib.cm as cm
from copy import deepcopy
import math

para_text = []
tfidf = []
clusters = []


def spacy_cleansing(doc):
    # for token in doc:
    #     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
    #             token.shape_, token.is_alpha, token.is_stop)
    doc = nlp(doc)
    lemmatized = []
    for token in doc:
        if not token.is_stop and token.is_alpha:
            lemmatized.append(token.lemma_)            
    return lemmatized


def art_to_para(article,threshold=70):
    #     For fragmentation of article text into paragraphs of word-count >= threshold
    #         Parameters :
    #         -------------
    #         article : str
    #         threshold : int, optional, default 50
    para_list = article.split('\n\n')
    if len(para_list) == 1 and len(para_list[0]) > 2.7*threshold :
        temp = para_list[0].split('. ')
        para_list = [" ".join(temp[:math.floor(len(temp)/2)]), " ".join(temp[math.floor(len(temp)/2):])]
    l = len(para_list)
    def para_thresholding(i=0, clustered_para=[]):
        temp = para_list[i]
        while len(temp.split(" ")) < threshold and i < l - 1:
            temp = temp + " " + para_list[i+1]
            i+=1
        clustered_para.append(temp) 
        if i == l-1:
            cluster_len = len(clustered_para[-1])
            if cluster_len < threshold and cluster_len > 1:
                clustered_para[-2] = clustered_para[-2] + " " + clustered_para[-1]
                del clustered_para[-1]
            return clustered_para
        else:
            return para_thresholding(i+1,clustered_para)
    return para_thresholding()

def load_and_break(keywords='rafale deal'):
    mypath = 'news-analyzer/data/'+keywords+'/news/'
    news_text_file = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    usr_input = input("For whole article, Press a \nFor paragraph segmentation, press p\n")
    bow = [] #........cleaned documents in list of lists
    if usr_input == 'a':
        for f in news_text_file:    
            text_article = pickle.load(open(f, 'rb'))
            temp = spacy_cleansing(text_article['content'])
            bow.append(temp)
    #         print ('article no : ',news_text_file.index(f), sorted(temp),'\n')
    else:
        para_count_in_art = []
        para_list = []
        for f in news_text_file:    
            text_article = pickle.load(open(f, 'rb'))
            temp = art_to_para(text_article['content'])
            para_list += temp
            para_count_in_art.append(len(temp))
            for t in temp:
                temp1 = spacy_cleansing(t)
                bow.append(temp1)
    store_file = {}
    store_file['para_count_in_art'] = para_count_in_art if usr_input != 'a' else 0
    store_file['para_list'] = para_list if usr_input != 'a' else 0
    store_file['clean_text_doclist'] = bow
#     if selection == 'a':
#         file = 'spacy_cleansing.pkl'
#     else:
#         file = 'spacy_cleansing_of_paras.pkl'
#     pickle.dump(store_file,open(file,'wb'))
    return store_file
processed_doc = load_and_break()
print('SUCCESS')

For whole article, Press a 
For paragraph segmentation, press p
 p


SUCCESS


In [6]:
# ---------------------------------- tfidf ---------------------------------------------------------------------


# processed_doc = pickle.load(open('spacy_cleansing_of_paras.pkl','rb'))
doc = processed_doc['clean_text_doclist']
wordSet = sorted(set().union(*doc))

doc_set_all = []
for docc in doc:
	doc_set = sorted(set().union(docc))
	doc_dict = dict.fromkeys(doc_set, 0)
	for word in docc:
		doc_dict[word] += 1 
	doc_set_all.append(doc_dict.copy())

# print ("wordSet  :   "," ".join(sorted(wordSet)))

def computeTF(tfDict,doc):
    doc_count = len(doc)
    for word, count in tfDict.items():
    	tfDict[word] = round((count+1)/doc_count,4)
#     print(str(tfDict),'\n')
    return tfDict

tfdoc = []
for i in range(len(doc)):
    tfdoc.append(computeTF(doc_set_all[i], doc[i]))


def computeIDF(doc_set_all,wordSet):
    import math
    idfDict = dict.fromkeys(wordSet, 0)
    N = len(doc_set_all)
    for word in wordSet:
        for i in range(N):
        	if word in doc_set_all[i].keys():
        		idfDict[word] += 1
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / val)
                        
    return idfDict        


idfs = computeIDF(doc_set_all,wordSet)

def computeTFIDF(tfdoc, idfs,threshold=0.01):
    tfidf = {}
    new_wordset = [] #for thresholding
    for word, val in tfdoc.items():
        ti = round(val*idfs[word],4)
        if ti > threshold:
            tfidf[word] = ti #------rounded for error handeling of data(float64)
            new_wordset.append(word)
#     print(str(tfidf),'\n')
#     print(str(new_wordset),'\n')
    return tfidf,new_wordset

def main_tfidf(threshold=0.01):
    tfidf = []
    new_wordSet = []
    for t in tfdoc:
        temp = computeTFIDF(t, idfs,threshold)
        new_wordSet += temp[1]
        tfidf.append(temp[0])
    wordSet = sorted(set().union(new_wordSet))
    tfidf_wordSet = [dict.fromkeys(wordSet, 0)]*len(doc)
    i=0 
    tfidf_final_struc = [] 
    for wrdF in tfidf_wordSet: 
        temp = tfidf[i] 
        # print(temp, wrdF) 
        tfidf_final_struc.append(dict(wrdF, **temp)) 
        i += 1
    df = pd.DataFrame(tfidf_final_struc)
    # df.to_csv('tfidf_of_paras.csv') 
    return df, tfidf


df, tfidf = main_tfidf()
print ('SUCCESS')

SUCCESS


In [7]:
#------------------------------------------- k clustering 2 --------------------------------------------------------
def main_clustering():
    # https://www.kaggle.com/andyxie/k-means-clustering-implementation-in-python
    # df=pd.read_csv('tfidf_of_paras.csv')
    X = df.iloc[:,1:].values
    X = np.nan_to_num(X)
    K=5
    m, feat = X.shape
    mean = np.mean(X, axis = 0)
    std = np.std(X, axis = 0)
    centers = np.random.randn(K,feat)*std + mean
    centers_old = np.zeros(centers.shape) # to store old centers
    centers_new = deepcopy(centers) # Store new centers
    clusters = np.zeros(m)
    distances = np.zeros((m,K))

    error = np.linalg.norm(centers_new - centers_old)

    # When, after an update, the estimate of that center stays the same, exit loop
    while error != 0:
        # Measure the distance to every center
        for i in range(K):
            distances[:,i] = np.linalg.norm(X - centers[i], axis=1)
        # Assign all training data to closest center
        clusters = np.argmin(distances, axis = 1)

        centers_old = deepcopy(centers_new)
        # Calculate mean for every cluster and update the center
        for i in range(K):
            centers_new[i] = np.mean(X[clusters == i], axis=0)
        error = np.linalg.norm(centers_new - centers_old)
    # centers_new 
    clusters
    cluster  = []
    for j in range(K):
        cluster.append(np.where(clusters == j))
    return cluster

cluster = main_clustering()
# print (cluster)
for single in cluster:
    print('==================================================================================================== \n\n\n')
    print( '\n\n\n','--------------------CLUSTER NUMBER {} WITH {}-PARA-----------:\n\n\n  '.format(cluster.index(single)+1, len(list(single[0]))))
    for i in list(single)[0]:
        print (i, '\n', processed_doc['para_list'][i], '\n\n',tfidf[i], '\n')







 --------------------CLUSTER NUMBER 1 WITH 88-PARA-----------:


  
2 
 Though Golay did not contest the polls, the party rank and file want him to become the next chief minister, the sources said. The Governor is learnt to be consulting legal experts about implications of inviting Golay to take oath as the chief minister, given that he was convicted to a year's imprisonment in a corruption case and had served the prison term. He was also disqualified as MLA in 2017. The SKM unseated the Sikkim Democratic Front (SDF) government led by five-time chief minister Pawan Kumar Chamling. 

 {'Chamling': 0.1164, 'Democratic': 0.0825, 'Golay': 0.1548, 'Governor': 0.1033, 'Kumar': 0.0796, 'MLA': 0.1164, 'Pawan': 0.1033, 'SDF': 0.1033, 'SKM': 0.1033, 'Sikkim': 0.1033, 'case': 0.0467, 'chief': 0.0985, 'consult': 0.1033, 'contest': 0.0902, 'convict': 0.1164, 'corruption': 0.0482, 'disqualify': 0.1164, 'expert': 0.0679, 'file': 0.0404, 'give': 0.039, 'government': 0.0137, 'implication': 0.1033

/home/gayar/.virtualenvs/jupyter/lib/python3.6/site-packages/ipykernel_launcher.py:43: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
